In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [51]:
# Вот здесь – эмитенты, там нужно отобрать именно страны
data_entity =  export_from_RISKCUSTOM('select * from "RISKACCESS"."umdapCBondsEmitents"')
# Вот здесь – сами рейтинги, нас интересуют long term foreign currency от трех агентств: Moody’s, Standarg & Poors (S&P), и Fitch Ratings.
data_rating = export_from_RISKCUSTOM('''select * from "RISKACCESS"."umdapCBondsEmitentRatings" where "agency_name_eng" in ('S&P Global Ratings', 'Moody''s Investors Service', 'Fitch Ratings') ''')
data_old = pd.read_excel('Country retings check.xlsx')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [184]:
# id всех стран - эмитентов
countries_df = data_entity.loc[(data_entity.is_country==1)&(data_entity.is_emitent==1), ['country_name_eng','id']].drop_duplicates()
countries_id_list = countries_df['id'].tolist()

In [217]:
# выбор нужного рейтинга
data_rating_LT_foreign = data_rating[data_rating.scale_name_eng.str.lower().apply(lambda x: 'foreign' in x.lower())]
# после выбора нужного рейтинга осталось 3 организации
assert data_rating_LT_foreign.agency_name_eng.nunique() == 3
# остался только долгосрочный рейтинг
assert ([x for x in data_rating_LT_foreign.scale_name_eng.unique() if 'LT' in x] == data_rating_LT_foreign.scale_name_eng.unique()).all() == np.True_
# выбор стран в качестве эмитента
data_rating_LT_foreign_countries = data_rating_LT_foreign[data_rating_LT_foreign.emitent_id.isin(countries_id_list)]
# выбор максимальной даты
data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_name_eng','agency_name_eng'])['updating_date'].transform(max)
data_rating_LT_foreign_countries_date = data_rating_LT_foreign_countries[data_rating_LT_foreign_countries.max_updating_date==data_rating_LT_foreign_countries.updating_date]

# pivot
data_rating_work = data_rating_LT_foreign_countries_date[['emission_emitent_name_eng','agency_name_eng','updating_date','forecast_name_eng', 'scale_point_id','scale_point_name']]
data_rating_work_pivot = data_rating_work.pivot_table(index='emission_emitent_name_eng', columns='agency_name_eng', values=['updating_date','forecast_name_eng','scale_point_name'], aggfunc=lambda x: list(x)[0]).reset_index()
# rename columns
columns_list = data_rating_work_pivot.columns.tolist()
new_names_dict = {}
for x in columns_list:
    if x[0] == 'forecast_name_eng':
        new_names_dict[x] = str(x[1].split(' ')[0]) + '_outlook'
    if x[0] == 'scale_point_name':
        new_names_dict[x] = str(x[1].split(' ')[0])
    if x[0] == 'updating_date':
        new_names_dict[x] = str(x[1].split(' ')[0]) + '_date'
data_rating_work_pivot.columns = ['Countries'] + list(new_names_dict.values())

# mergw old data 
data_new = data_old.loc[:,:'Countries'].merge(data_rating_work_pivot,how='outer', on='Countries').sort_values(['Fitch_date',	'Moody\'s_date',	'S&P_date'])


C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9376\1422236817.py:10: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_name_eng','agency_name_eng'])['updating_date'].transform(max)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9376\1422236817.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_rating_LT_foreign_countries['max_updating_date'] = data_rating_LT_foreign_countries.groupby(dropna=False, by=['emission_emitent_n

In [164]:
data_rating_LT_foreign.scale_name_eng.unique()

array(['LT Int. Scale (foreign curr.)', 'LT- foreign currency',
       'Foreign Currency LT'], dtype=object)

In [189]:
data_new

,Code,Countries,Fitch_outlook,Moody's_outlook,S&P_outlook,Fitch,Moody's,S&P,Fitch_date,Moody's_date,S&P_date
62,ML,Mali,None,Stable,None,Withdrawn,Caa2,NR,2016-08-24 09:50:52,2024-06-12 09:28:24,2021-03-17 16:35:58
84,RU,Russia,None,None,None,Withdrawn,Withdrawn,NR,2022-03-27 16:48:58,2022-03-31 16:36:32,2022-04-11 07:42:47
10,BY,Belarus,None,Stable,None,Withdrawn,C,NR,2023-10-09 09:10:58,2023-06-05 09:55:41,2023-09-11 10:00:02
5,AU,Australia,Stable,Stable,Stable,AAA,Aaa,AAA,2023-11-10 08:47:55,2024-10-10 09:12:59,2024-09-26 10:09:05
101,TH,Thailand,Stable,Stable,Stable,BBB+,Baa1,BBB+,2023-11-14 09:51:42,2024-04-12 10:24:00,2023-11-29 09:08:06
...,...,...,...,...,...,...,...,...,...,...,...
86,SV,Salvador,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
91,NaN,Slovakia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
92,NaN,Slovenia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
102,NaN,Togo,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [190]:
data_new.to_excel(f'{str(date.today())}_Country_retings_check.xlsx',index=False)

In [218]:
data_new_null = data_new[(data_new.loc[:,'Fitch_outlook':].isna()).all(axis=1)]
data_new_null

,Code,Countries,Fitch_outlook,Moody's_outlook,S&P_outlook,Fitch,Moody's,S&P,Fitch_date,Moody's_date,S&P_date
1,DZ,Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
17,NaN,Cambodia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
24,CI,Cote d'Ivoire,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
26,NaN,Croatia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
33,NaN,Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
34,FJ,Fiji,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
54,NaN,Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
57,LR,Liberia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
60,MK,Macedonia,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
63,MH,Marshall Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT


In [194]:
data_new_null.shape[0]

19

In [219]:
data_wikirating = pd.read_excel('c.wikirating_com.xlsx')
data_wikirating_work = data_wikirating[['Country', 'S&P', 'Moody\'s', 'Fitch', 'Last update']]
data_wikirating_merge = data_new_null.merge(data_wikirating_work, how='left', left_on='Countries', right_on='Country')

In [226]:

replace_dict = {'r.w.':np.nan, 'n.r.':np.nan}
data_wikirating_merge[['S&P_y','Moody\'s_y','Fitch_y']] = data_wikirating_merge[['S&P_y','Moody\'s_y','Fitch_y']].replace(replace_dict)
data_wikirating_merge[['S&P_x','Moody\'s_x','Fitch_x']] = data_wikirating_merge[['S&P_y','Moody\'s_y','Fitch_y']] 

data_wikirating_merge.loc[~data_wikirating_merge['S&P_x'].isna(), 'S&P_date'] = data_wikirating_merge.loc[~data_wikirating_merge['S&P_x'].isna(), 'Last update']
data_wikirating_merge.loc[~data_wikirating_merge['Moody\'s_x'].isna(), 'Moody\'s_date'] = data_wikirating_merge.loc[~data_wikirating_merge['Moody\'s_x'].isna(), 'Last update']
data_wikirating_merge.loc[~data_wikirating_merge['Fitch_x'].isna(), 'Fitch_date'] = data_wikirating_merge.loc[~data_wikirating_merge['Fitch_x'].isna(), 'Last update']
data_new_null_new = data_wikirating_merge.iloc[:,:-5].rename(columns={'S&P_x':'S&P','Moody\'s_x':'Moody\'s','Fitch_x':'Fitch'})

In [241]:
data_full = pd.concat([data_new[(~data_new.loc[:,'Fitch_outlook':].isna()).any(axis=1)],data_new_null_new])
# .to_excel(f'{str(date.today())}_Country_retings_check_m.xlsx',index=False)

In [245]:
name_counries = pd.read_csv('artlebedev_ru.txt', sep='	', usecols=['english','alpha2'])
name_counries_merge = data_full.merge(name_counries, how='left', left_on='Countries', right_on='english')
name_counries_merge.loc[name_counries_merge.Code.isna(), 'Code'] = name_counries_merge.loc[name_counries_merge.Code.isna(), 'alpha2']
name_counries_merge

,Code,Countries,Fitch_outlook,Moody's_outlook,S&P_outlook,Fitch,Moody's,S&P,Fitch_date,Moody's_date,S&P_date,english,alpha2
0,ML,Mali,None,Stable,None,Withdrawn,Caa2,NR,2016-08-24 09:50:52,2024-06-12 09:28:24,2021-03-17 16:35:58,Mali,ML
1,RU,Russia,None,None,None,Withdrawn,Withdrawn,NR,2022-03-27 16:48:58,2022-03-31 16:36:32,2022-04-11 07:42:47,NaN,NaN
2,BY,Belarus,None,Stable,None,Withdrawn,C,NR,2023-10-09 09:10:58,2023-06-05 09:55:41,2023-09-11 10:00:02,Belarus,BY
3,AU,Australia,Stable,Stable,Stable,AAA,Aaa,AAA,2023-11-10 08:47:55,2024-10-10 09:12:59,2024-09-26 10:09:05,Australia,AU
4,TH,Thailand,Stable,Stable,Stable,BBB+,Baa1,BBB+,2023-11-14 09:51:42,2024-04-12 10:24:00,2023-11-29 09:08:06,Thailand,TH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,SV,Salvador,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
111,SK,Slovakia,NaN,NaN,NaN,A,A2,A+,2024-02-09 00:00:00,2024-02-09 00:00:00,2024-02-09 00:00:00,Slovakia,SK
112,SI,Slovenia,NaN,NaN,NaN,A,A3,AA-,2024-07-05 00:00:00,2024-07-05 00:00:00,2024-07-05 00:00:00,Slovenia,SI
113,TG,Togo,NaN,NaN,NaN,NaN,B3,B,NaT,2024-09-27 00:00:00,2024-09-27 00:00:00,Togo,TG


In [247]:
name_counries_merge.to_excel(f'{str(date.today())}_Country_retings_check_m.xlsx',index=False)